In [ ]:
import sys, os
ROOT = os.path.abspath(os.curdir)
PARENT = os.path.dirname(ROOT)

if PARENT not in sys.path:
    sys.path.insert(0, PARENT)

In [1]:

import numpy as np
import scipy as sp
import h5py

import src.SimData as sim_data
import src.Simulator as sim_system
import src.Optimizer as opt

In [2]:


###* Input files and function

reactions_file = "../reactions/reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "../Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"

In [3]:


def func_optimization(params):
    
    SF_1, SF_2, SF_3, SF_4, SF_5, SF_6, SF_7, SF_8 = params
    
    A = 2.777e-02
    B = 7.792e-04
    E = 1.627e+01
    
    nu_d_mod = lambda T: 1e15 * (A + B * np.exp(E/(const_dict['R'] * T)))
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    
    {"id": 32, "rate": None, "model_dict": {"SF": SF_1}},
    {"id": 35, "rate": None, "model_dict": {"SF": SF_2}},
    
    {"id": 33, "rate": None, "model_dict": {"SF": SF_3}},
    {"id": 34, "rate": None, "model_dict": {"SF": SF_4}},
    
    {"id": 36, "rate": None, "model_dict": {"SF": SF_5}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_6}},
    
    {"id": 38, "rate": None, "model_dict": {"SF": SF_7}},
    {"id": 39, "rate": None, "model_dict": {"SF": SF_8}},
    ]
    
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)

exp_file = os.path.join(output_folder_path, exp_data_file)
sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [4]:

if __name__ == '__main__':
    optimizer = opt.Optimizer(sim, func_optimization, loss_function)

    global_bounds = [
                    (1e-3, 5e-2), (5e-2, 0.5), (1e-3, 9e-2), 
                    (1e-3, 9e-2), (6e-2, 6e-1), (9e-3, 9e-2),
                    (5e-3, 9e-2), (5e-2, 0.3)
                    ]

    config = {
        "bounds": global_bounds,
        "de_maxiter": 3,
        "de_num_iterations": 1,
    }
    
    best_local, best_local_loss = optimizer.hybrid_optimization_searchV2(config)

# differential_evolution step 50: f(x)= 0.041662857183863554


differential_evolution step 1: f(x)= 0.03858644361587933
differential_evolution step 2: f(x)= 0.03858644361587933
differential_evolution step 3: f(x)= 0.03856350926045422
differential_evolution step 4: f(x)= 0.03854901202378275
differential_evolution step 5: f(x)= 0.03854166600508579
differential_evolution step 6: f(x)= 0.03854166600508579
differential_evolution step 7: f(x)= 0.03854166600508579
top_k_mod:  36
res:  0.038533558000600876 [0.01940521 0.45341856 0.05601811 0.04320305 0.57830548 0.05506907
 0.00559461 0.1982866 ] [0.01959608 0.41794793 0.05304598 0.04256952 0.5908371  0.05338653
 0.00654808 0.20482056]
res:  0.03854574719045397 [0.01874063 0.18714673 0.00312948 0.03545555 0.55205504 0.02105857
 0.00509997 0.24674397] [0.01862916 0.18277738 0.00305642 0.03462776 0.55458994 0.02056691
 0.00563436 0.2409832 ]
res:  0.03853819664068984 [0.01767579 0.40485284 0.08086422 0.03938382 0.52679419 0.06066778
 0.00547092 0.19338651] [0.01801354 0.3955814  0.07901237 0.0384819  0.51473

In [5]:
print(best_local)
print(best_local_loss)

             message: Optimization terminated successfully.
             success: True
                 fun: 0.03854166600508579
                   x: [ 1.960e-02  4.179e-01  5.305e-02  4.257e-02
                        5.908e-01  5.339e-02  6.548e-03  2.048e-01]
                 nit: 7
                nfev: 960
          population: [[ 1.960e-02  4.179e-01 ...  6.548e-03  2.048e-01]
                       [ 2.001e-02  3.185e-01 ...  2.489e-02  1.773e-01]
                       ...
                       [ 2.023e-02  3.828e-01 ...  5.731e-02  2.666e-01]
                       [ 1.998e-02  2.202e-01 ...  3.852e-02  2.095e-01]]
 population_energies: [ 3.854e-02  3.864e-02 ...  3.880e-02  3.873e-02]
             message: Optimization terminated successfully.
             success: True
                 fun: 0.038533558000600876
                   x: [ 1.941e-02  4.534e-01  5.602e-02  4.320e-02
                        5.783e-01  5.507e-02  5.595e-03  1.983e-01]
                 nit: 1
     